## Research And Generate
RAG System written using Ollama and LLangchain Libraries

Use this runtime for online testing
https://colab.research.google.com/drive/1s9tpx736ohbZu0STumiba3-noPSV0Wdg?usp=sharing

1. Installing Required Libraries, in this case llangchain for Enconding stuff

In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchainhub gpt4all langchain-chroma pypdf
!pip install -U langchain-chroma

Importing Installed Libraries

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()

/tmp/ipykernel_15617/726452791.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma()


Loading an exmaple Website

In [2]:
loader = PyPDFLoader("nsbm_foc_data.pdf")
pages = []

async for page in loader.alazy_load():
    pages.append(page)

Splitting the downloaded data and storing them in a vectorized database object, Doing this only once is enough

In [3]:
vectorstore = Chroma.from_documents(documents=pages, embedding=GPT4AllEmbeddings())

Testing similarity search

In [21]:
question = "What is the dean's message?"
docs = vectorstore.similarity_search(question)
len(docs)

4


### *Okay  now the Research part is done*
Lets focus on generating part now

Installing libraries required to run a LLM locally.

In [ ]:
!pip install transformers

In [1]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import pipeline



2025-03-01 08:25:36.037779: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/zerone/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/zerone/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Testing Summerizer

In [ ]:
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# text = """
# NSBM (National School of Business Management) is a leading university located in Sri Lanka. It offers a range of undergraduate and postgraduate programs in various fields such as Business, Information Technology, and Engineering. 
# The faculty is composed of highly qualified professionals who are experts in their respective fields. Students benefit from state-of-the-art facilities, interactive learning methods, and a diverse cultural environment. 
# The university also emphasizes industry collaborations, providing students with ample opportunities for internships and job placements after graduation.
# """

# # Get the summary of the text
# summary = summarizer(docs, max_length=150, min_length=50, do_sample=False)

# # Output the summary
# print("Summary:", summary[0]['summary_text'])

In [ ]:
!pip install sentencepiece


Let's Download the model now

In [ ]:
!ls
!nvidia-smi
!nvcc --version

Setting the model parameters

In [6]:
model_path = "cognitivecomputations/Dolphin3.0-Llama3.2-3B"
model = AutoModelForCausalLM.from_pretrained(model_path,  device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/zerone/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Testing the model capabilities

Let's Combine the context and the question

In [ ]:
context = "\n".join([doc.page_content for doc in docs])
token_counts = sum([len(tokenizer.encode(doc.page_content)) for doc in docs])

# According to the internet, the maximum token count of LLAMA3.2 is 4096, 
# So just to be safe, lets truncate the context input to 2000 tokens
print(token_counts)

# Truncate context to 2000 tokens
tokens = tokenizer.encode(context, add_special_tokens=False)
truncated_tokens = tokens[:2000]
context = tokenizer.decode(truncated_tokens, skip_special_tokens=True)

prompt = f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"

input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(**input_ids, max_new_tokens=4000)
answer = tokenizer.decode(output[0], skip_special_tokens=True)

print(answer)

## Implementing Memory with Langchain [TODO]

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessageState):
    system_pormpt = (
        "You are a helpful assistant",
        "Your name is NSBM Chat"
    )
    message = [SystemMessage(content=system_prompt)] + state["messages"]
    response = mode.invoke(message)
    return {"message": message}

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Testing Model's Capabilities with ROUGE Scores

In [ ]:
!pip install rouge_score

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer
from rouge_score import rouge_scorer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def initialize_vectorstore():
    """Initialize ChromaDB"""
    vectorstore = Chroma()
    return vectorstore

def load_pdf(file_path):
    """Load the PDF"""
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

def load_model(model_path):
    """Load the Model."""
    model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

def process_text(pages, tokenizer, max_tokens=2000):
    """Truncating"""
    context = "\n".join([page.page_content for page in pages])
    tokens = tokenizer.encode(context, add_special_tokens=False, truncation=True, max_length=max_tokens)
    return tokenizer.decode(tokens, skip_special_tokens=True)

def generate_answer(model, tokenizer, context, question):
    """Generate response"""
    prompt = f"Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_new_tokens=400)
    return tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()

def ROUGE(reference, gpt):
    """Calculate ROUGE score"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference, gpt)

def main():
    vectorstore = initialize_vectorstore()
    pages = load_pdf("nsbm_foc_data.pdf")
    model, tokenizer = load_model("cognitivecomputations/Dolphin3.0-Llama3.2-3B")
    context = process_text(pages, tokenizer)

    questionArray = [
        "Who is the Dean of NSBM FOC?",
        "How to contact NSBM for new Enrollments?",
        "What is the dean's message about? Answer in a paragraph"
    ]
    
    referenceAnswerArray = [
        "Dean of the NSBM's Faculty of Computing is Dr. Rasika Ranaweera.",
        "Send an email to inquiries@nsbm.ac.lk.",
        """The Dean's message welcomes students to the Faculty of Computing at NSBM Green University, emphasizing innovative and industry-relevant education. 
        It highlights the well-qualified faculty and strong student support services aimed at preparing students for successful careers. 
        The message encourages students to work hard, set high standards, and take responsibility for their learning. Ultimately, it expresses the university’s goal 
        of developing well-rounded individuals who will succeed in life and serve as ambassadors for NSBM."""
    ]

    gptAnswerArray = []
    scoreArray = []

    for question in questionArray:
        gptAnswer = generate_answer(model, tokenizer, context, question)
        gptAnswerArray.append(gptAnswer)

    for i, (reference, gpt) in enumerate(zip(referenceAnswerArray, gptAnswerArray)):
        print(f"Calculating ROUGE Scores for Question {i+1}")
        print("Reference:", reference)
        print("GPT:", gpt)
        scores = ROUGE(reference, gpt)
        print(f"Scores: {scores}")
        print("-----------------------------")
    print("Testing Completed !  Displaying Scores...")
    for i, score in enumerate(scoreArray):
        print("----------------------------")
        print(f"Scores for Question {i+1}:")
        print(f"Question= {questionArray[i]}")
        print(f"Reference Answer = {referenceAnswerArray[i]}")
        print(f"Model Answer = {gptAnswerArray[i]}")
        print(f"ROUGE Scores: {score}")
        print("-----------------------------")

if __name__ == "__main__":
    main()
